# Grafón: Qué electivas me conviene cursar?

Este grafo se encarga de analizar "gente que haya tenido experiencias facultativas similares", sin importar en que año o cuatrimestre fue. 

La idea es que si me fue parecido en varias materias a alguna otra persona, y esa persona curso alguna electiva que yo no, entonces esa materia es una buena candidata para mí.

Por ejemplo: Por más que Juan se haya recibido el año pasado, es una persona que se parece mucho a mí porque nos sacamos las mismas notas en muchas materias. Probablemente las electivas que Juan eligió, me sirvan a mí como guía.

## Cómo es el grafo?

El grafo analizado va a ser un grafo simple, y no un multigrafo: entre cada par de alumnos solo puede haber una arista. Manejamos la cardinalidad en el peso, en vez de tener múltiples aristas.

- Nodos: alumnos
- Aristas: conectar dos alumnos que hayan cursado misma materia y "les fue parecido".
    - A los dos nos fue muy bien (nos sacamos entre 8 y 10)    
    - A los dos nos fue más o menos (nos sacamos un 6 o un 7)
    - A los dos nos fue mal (nos sacamos un 4 o un 5)
- Peso: La inversa* de la cantidad de materias en donde somos similares

\* Calculamos la inversa porque mientras mas similares son, menor peso queremos que haya entre la arista, para que más cercanos esten

### Dos formas de armar el análisis
// ToDo: que hacemos aca? Elegimos uno, o presentamos ambos en el mismo notebook?
// Miti miti? la parte de abajo mas enfocada en solo electivas¨
- sólo correr sobre materias electivas
- correr sobre todas las materias, pero filtrar el output por electivas

In [ ]:
import pandas as pd
import utils

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
# Armamos las categorias de que les haya hido parecido a dos alumnos
categories = {
    4: 0,
    5: 0,
    6: 1,
    7: 1,
    8: 2,
    9: 2,
    10: 2
}

# Sacamos materias en final y a cursar
df_alumnos = df[df['materia_nota'] >= 4]

# Sacamos gente que no le pone la nota a su fiubamap y deja que se saco (casi) todos 4s directamente
df_alumnos['mediana'] = df_alumnos.groupby('Padron')['materia_nota'].transform('median')
df_alumnos = df_alumnos[df_alumnos['mediana'] > 5]

df_alumnos['nota_categoria'] = df_alumnos['materia_nota'].apply(lambda x: categories[x])

# Juntamos el grafo con si mismo para tener la similiritud entre cada par de padrones
df_simil_sinagg = pd.merge(df_alumnos, df_alumnos, on=['materia_id', 'nota_categoria'])
df_simil_sinagg = df_simil_sinagg[df_simil_sinagg['Padron_x'] != df_simil_sinagg['Padron_y']]
df_simil_sinagg = df_simil_sinagg.reset_index()

df_simil_sinagg = df_simil_sinagg[['materia_id', 'nota_categoria', 'Padron_x', 'materia_nota_x', 'Padron_y', 'materia_nota_y']]

# Unificar aristas ( con el objetivo de no tener pocos nodos y millones de aristas )
df_simil = df_simil_sinagg.groupby(['Padron_x', 'Padron_y']).agg(cant_materias_similares=('materia_id', 'count'))
df_simil = df_simil.reset_index()

df_simil['Padron_min'] = df_simil[['Padron_x', 'Padron_y']].min(axis=1)
df_simil['Padron_max'] = df_simil[['Padron_x', 'Padron_y']].max(axis=1)
df_simil = df_simil.drop_duplicates(['Padron_min', 'Padron_max']).reset_index()

# ToDo: fijarse si elevar a un K acá
df_simil['inv_cant_materias_similares'] = df_simil['cant_materias_similares'].max() - df_simil['cant_materias_similares'] + 1
df_simil[['Padron_x', 'Padron_y', 'cant_materias_similares', 'inv_cant_materias_similares']]


# Nota: Al no tener en cuenta el factor temporal en este grafo (en que cuatri cursó cada alumno), esta vez no se aplica el filtro de "sacar todos los alumnos que no setean el cuatrimestre"
# Ese filtro es considerable, por lo que tiene sentido que este grafo sea bastante mas grande que grafazo
display(df_simil.sample(3))

In [ ]:
# Veamos los padrones más parecidos entre sí
df_simil.sort_values('cant_materias_similares', ascending=False).head(10)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_pandas_edgelist(df_simil, 
                            source='Padron_x', 
                            target='Padron_y', 
                            edge_attr='inv_cant_materias_similares',
                            create_using=nx.Graph())

utils.stats(G)
utils.plot(G, edge_width=0.0005)

## Comunidades

\# ToDo: explicar que pretendemos ver acá
\# ToDo: Explicar que a diferencia de grafazo, aca estamos viendo cosas un poco mas subjetivas, entonces no tenemos validación de homofilia para hacer

In [ ]:
from networkx.algorithms import community

louvain = community.louvain_communities(G, weight='inv_cant_materias_similares')
utils.plot_communities(G, louvain)

In [ ]:
# ToDo: Ver de agregarle logica a las subcomunidades, para que no se sienta que es nada mas correr el algoritmo generico original hasta que nos de algo que nos guste

In [ ]:
from config import PADRON, CARRERA
from utils import plan_estudios

def nestedsearch(el, lst_of_sets):
    return list(filter(lambda lst: el in lst, lst_of_sets))[0]

def materias_padron(padron):
    return df[(df['Padron'] == padron) & (df['materia_nota'] >= 4)]['materia_id'].values

def sugerir_electivas(padron):
    min_alumnos, max_alumnos = 6, 20
    max_iteraciones = 25

    i = 0
    grupo = []
    for i in range(max_iteraciones):
        louvain = community.louvain_communities(G, weight='inv_cant_materias_similares', resolution=1+(i*0.01))
        comunidad = nestedsearch(padron, louvain)
        if min_alumnos <= len(comunidad) <= max_alumnos:
            grupo = comunidad
            break
        elif not grupo or (len(comunidad) >= max_alumnos and (len(comunidad) - max_alumnos <= len(grupo) - max_alumnos)):
            grupo = comunidad
        i+=1
    
    df_sugerencias = df_alumnos[df_alumnos['Padron'].isin(grupo)].groupby('materia_id').agg(cant_alumnos_similares=('materia_id', 'count'))
    df_sugerencias = df_sugerencias[~df_sugerencias.index.isin(materias_padron(padron))]
    
    df_materias = pd.read_json(plan_estudios(CARRERA))
    df_sugerencias = pd.merge(df_sugerencias, df_materias, left_on='materia_id', right_on="id")
    df_sugerencias = df_sugerencias[df_sugerencias['categoria'] == 'Materias Electivas']
    df_sugerencias = df_sugerencias[['id', 'materia', 'creditos', 'cant_alumnos_similares']].sort_values('cant_alumnos_similares', ascending=False)
    return df_sugerencias.reset_index(drop=True)

print(f"Top 10 electivas sugeridas a {PADRON}")
electivas = sugerir_electivas(PADRON)
electivas.head(10)